In [2]:
# Import Libraries 
import os
import numpy as np
import pandas as pd
from nltk.stem.lancaster import LancasterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk import word_tokenize

In [11]:
# import data
data = pd.read_csv("C:/Users/JRF/Desktop/CGCB/Data/FAQ.csv")

In [12]:
#view data
data.head()

,Question,Answer,Class
0,Hi,I am here to help your FAQ. Kindly type the ke...,Genral
1,What are the contact details of the Department...,Department of Administrative Reforms and Publi...,contact
2,Where can the grievances be sent?,The grievances can be sent to The Department ...,sent
3,How do I lodge the grievance?,The grievances can be lodged online on . In ca...,lodge
4,What happens when I lodge the grievance?,The grievance is acknowledged online or by pos...,NaN


In [6]:
#Import nltk package
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JRF\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
questions = data['Question'].values
answers = data['Answer'].values

In [14]:
stemmer = LancasterStemmer()
def cleanup(sentence):
    word_tokens = word_tokenize(sentence)
    stemmed_words = [stemmer.stem(word) for word in word_tokens]
    return ' '.join(stemmed_words)
questions_stemmed = [cleanup(question) for question in questions]

In [15]:

tfv = TfidfVectorizer(min_df = 1, stop_words = 'english')

In [16]:
questions_vectors = tfv.fit_transform(questions_stemmed)

In [18]:
# For a new question:
temp_question = 'any other queries'
temp_question_cleaned = cleanup(temp_question)
temp_question_vectorized = tfv.transform([temp_question_cleaned])

In [19]:
# Finding the similarity with all other questions:
cosines_similarities = []
for i in range(questions_vectors.shape[0]):
    cosines_similarities.append(cosine_similarity(questions_vectors[i], temp_question_vectorized)[0][0])
    

In [20]:
def Answer(question):
    question_cleaned = cleanup(question)
    question_vector = tfv.transform([question_cleaned])
    cosine_similarities = []
    for i in range(questions_vectors.shape[0]):
        cosine_similarities.append(cosine_similarity(questions_vectors[i], question_vector))
    index = np.argmax(np.asarray(cosine_similarities))
    answer = answers[index]
    return index, answer
Answer(temp_question)

(0, 'I am here to help your FAQ. Kindly type the keyword here ')

In [21]:
answers[np.argmax(np.asarray(cosines_similarities))]

'I am here to help your FAQ. Kindly type the keyword here '

In [22]:
def bot():
    while(True):
        user_question = input()
        if user_question == 'Q':
            break
        else:
            # cleaning the question:
            user_question_cleaned = cleanup(user_question)
            # vectorizing the question:
            user_question_vector = tfv.transform([user_question_cleaned])
            cosines_similarities = []
            for i in range(questions_vectors.shape[0]):
                cosines_similarities.append(cosine_similarity(questions_vectors[i],user_question_vector)[0][0])
            answer = answers[np.argmax(np.asarray(cosines_similarities))]
            print("Answer: ",answer)
        print('')
    return 'Bye'

In [ ]:
bot()

hi
Answer:  I am here to help your FAQ. Kindly type the keyword here 

time
Answer:  Sixty (60) days. In case of delay an interim reply with reasons for delay is required to be given.

contact
Answer:  Department of Administrative Reforms and Public Grievances 5th floor Sardar Patel Bhavan Sansad Marg New Delhi 110001 

website
Answer:  I am here to help your FAQ. Kindly type the keyword here 



In [ ]:
# Finding Accuracy:
correct_answers = 0
wrong_answers = 0
import datetime
a = datetime.datetime.now()

for i in range(len(questions)):
    question = questions[i]
    answer_index , _= Answer(question)
    if i == answer_index:
        correct_answers += 1
    else:
        wrong_answers += 1

b = datetime.datetime.now()
time_taken = (b - a).total_seconds()
print('Accuracy: ', (correct_answers/ len(questions)))
print("The time taken for {} questions is {} seconds:".format(len(questions), time_taken))